<a href="https://colab.research.google.com/github/glennamaria/Projects/blob/main/Delinquency%20prediction%20and%20console.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
import joblib

In [ ]:
from google.colab import drive
import pandas as pd
import os

from google.colab import drive
drive.mount('/content/drive')

path="/content/drive/MyDrive/SME.csv"
df= pd.read_csv(path)
df

In [ ]:
label_encoders = {}
for column in ['Gender', 'Branch Name']:  # Encode categorical variables
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

    print("Encoded DataFrame:")
print(df)

# To get the encoded values for each column, you can do:
encoded_gender = df['Gender'].unique()  # Unique encoded values for Gender
encoded_branch = df['Branch Name'].unique()  # Unique encoded values for Branch Name

print("\nEncoded values for Gender:")
print(encoded_gender)

print("\nEncoded values for Branch Name:")
print(encoded_branch)

In [ ]:
CHANDAPUR_encoded = le.transform(['CHANDAPUR'])[0]  # Returns the encoded value for 'Mysore'
print("\nEncoded value for 'CHANDAPUR':", CHANDAPUR_encoded)

In [ ]:
df = df.dropna()  # Drop rows with missing values for simplicity

# Define features and target
X = df.drop('Deliquency', axis=1)  # Replace with actual target column name
y = df['Deliquency']

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

In [ ]:
smote = SMOTE()
X_resampled, y_resampled = smote.fit_resample(X_scaled, y)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [ ]:
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

In [ ]:
joblib.dump(model, 'loan_model.pkl')
joblib.dump(scaler, 'scaler.pkl')

In [ ]:
# Function to predict new entries
def predict_delinquency(loan_amount, age, net_income, cibil, gender, tenure, branch_name):
    # Load the model and scaler
    model = joblib.load('loan_model.pkl')
    scaler = joblib.load('scaler.pkl')

    # Prepare the input data
    data = pd.DataFrame([[loan_amount, age, net_income, cibil, gender, tenure, branch_name]],
                        columns=['LoanAmount', 'Age', 'NetIncome', 'CIBIL', 'Gender', 'Tenure', 'Branch Name'])

    # Encode categorical variables
    categorical_columns = ['Gender', 'Branch Name'] # Define the list of categorical columns
    for col in categorical_columns:
        # Check if the value exists in the label encoder's classes
        if gender not in label_encoders[col].classes_:
            # Handle the new value (e.g., ignore or assign a default value)
            # Here, we'll ignore for demonstration
            continue
        data[col] = label_encoders[col].transform(data[col])

    # Standardize the features
    data = scaler.transform(data)

    # Predict the delinquency
    prediction = model.predict(data)
    return 'Delinquent' if prediction[0] == 1 else 'Not Delinquent'

# Console application for new entries
if __name__ == "__main__":
    print("Loan Delinquency Prediction Console")
    loan_amount = float(input("Enter Loan Amount: "))
    age = float(input("Enter Age: "))
    net_income = float(input("Enter Net Income: "))
    cibil = float(input("Enter CIBIL Score: "))
    gender = input("Enter Gender (Male/Female): ")
    tenure = float(input("Enter Tenure (in months): "))
    branch_name = input("Enter Branch Name: ")

    prediction = predict_delinquency(loan_amount, age, net_income, cibil, gender, tenure, branch_name)
    print(f"Prediction: {prediction}")

Loan Delinquency Prediction Console
Enter Loan Amount: 310000
Enter Age: 49
Enter Net Income: 56300
Enter CIBIL Score: 0
Enter Gender (Male/Female): 0
